TASK : 1 LABEL INFERENCING

In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.5 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import argparse
import pandas as pd
import torch
import os

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report, \
    precision_recall_fscore_support, precision_score, recall_score

In [3]:
!pip install evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00


In [4]:
def get_dict(dataset_name):
    if dataset_name == 'CAMS':
        labels = {0: 'none', 1: 'bias', 2: 'job', 3: 'medication', 4: 'relation', 5: 'alienation'}
        num_labels = 6
    elif dataset_name == 'CLP':
        labels = {0: 'no', 1: 'yes'}
        num_labels = 2
    elif dataset_name == 'DR':
        labels = {0: 'no', 1: 'yes'}
        num_labels = 2
    elif dataset_name == 'dreaddit':
        labels = {0: 'no', 1: 'yes'}
        num_labels = 2
    elif dataset_name == 'Irf':
        labels = {0: 'no', 1: 'yes'}
        num_labels = 2
    elif dataset_name == 'loneliness':
        labels = {0: 'no', 1: 'yes'}
        num_labels = 2
    elif dataset_name == 'MultiWD':
        labels = {0: 'no', 1: 'yes'}
        num_labels = 2
    elif dataset_name == 'SAD':
        labels = {0: 'school', 1: 'financial',2: 'family', 3: 'social',4: 'work', 5: 'health',6: 'emotional', 7: 'everyday',8: 'other'}
        num_labels = 9
    elif dataset_name == 'swmh':
        labels = {0: 'depression', 1: 'suicide', 2: 'anxiety', 3: 'bipolar', 4: 'no mental'}
        num_labels = 5
    elif dataset_name == 't-sid':
        labels = {0: 'depression', 1: 'suicide', 2: 'ptsd', 3: 'control'}
        num_labels = 4
    else:
        raise Exception("ERROR! please choose the correct dataset!")

    return labels, num_labels

In [5]:
def calculate_f1(goldens, final_labels, dataset_name):
    golden_label = []
    output_label = []
    for golden, label in zip(goldens, final_labels):
        ref_an = golden.split("Reasoning:")[0]
        output_an = label.strip()

        if 'swmh' in dataset_name:
            if 'no mental' in output_an.lower():
                output_label.append(0)
            elif 'suicide' in output_an.lower():
                output_label.append(1)
            elif 'depression' in output_an.lower():
                output_label.append(2)
            elif 'anxiety' in output_an.lower():
                output_label.append(3)
            elif 'bipolar' in output_an.lower():
                output_label.append(4)
            else:
                raise Exception('Wrong label in predictions for {}'.format(dataset_name))

            if 'no mental' in ref_an.lower():
                golden_label.append(0)
            elif 'suicide' in ref_an.lower():
                golden_label.append(1)
            elif 'depression' in ref_an.lower():
                golden_label.append(2)
            elif 'anxiety' in ref_an.lower():
                golden_label.append(3)
            elif 'bipolar' in ref_an.lower():
                golden_label.append(4)
            else:
                output_label = output_label[:-1]

        elif dataset_name == 't-sid':
            if 'depression' in output_an.lower():
                output_label.append(2)
            elif 'suicide' in output_an.lower():
                output_label.append(1)
            elif 'ptsd' in output_an.lower():
                output_label.append(3)
            elif 'control' in output_an.lower():
                output_label.append(0)
            else:
                raise Exception('Wrong label in predictions for {}'.format(dataset_name))

            if 'depression' in ref_an.lower():
                golden_label.append(2)
            elif 'suicide or self-harm' in ref_an.lower():
                golden_label.append(1)
            elif 'ptsd' in ref_an.lower():
                golden_label.append(3)
            elif 'no mental' in ref_an.lower():
                golden_label.append(0)

        elif dataset_name in ['CLP', 'DR', 'dreaddit', 'loneliness', 'Irf', 'MultiWD']:
            if 'yes' in output_an.lower():
                output_label.append(1)
            elif 'no' in output_an.lower():
                output_label.append(0)
            else:
                raise Exception('Wrong label in predictions for {}'.format(dataset_name))

            if 'yes' in ref_an.lower():
                golden_label.append(1)
            elif 'no' in ref_an.lower():
                golden_label.append(0)

        elif dataset_name == 'SAD':
            if 'school' in output_an.lower():
                output_label.append(0)
            elif 'financial' in output_an.lower():
                output_label.append(1)
            elif 'family' in output_an.lower():
                output_label.append(2)
            elif 'social' in output_an.lower():
                output_label.append(3)
            elif 'work' in output_an.lower():
                output_label.append(4)
            elif 'health' in output_an.lower():
                output_label.append(5)
            elif 'emotional' in output_an.lower():
                output_label.append(6)
            elif 'everyday' in output_an.lower():
                output_label.append(7)
            elif 'other' in output_an.lower():
                output_label.append(8)
            else:
                raise Exception('Wrong label in predictions for {}'.format(dataset_name))

            if 'school' in ref_an.lower():
                golden_label.append(0)
            elif 'financial problem' in ref_an.lower():
                golden_label.append(1)
            elif 'family issues' in ref_an.lower():
                golden_label.append(2)
            elif 'social relationships' in ref_an.lower():
                golden_label.append(3)
            elif 'work' in ref_an.lower():
                golden_label.append(4)
            elif 'health issues' in ref_an.lower():
                golden_label.append(5)
            elif 'emotion turmoil' in ref_an.lower():
                golden_label.append(6)
            elif 'everyday decision making' in ref_an.lower():
                golden_label.append(7)
            elif 'other' in ref_an.lower():
                golden_label.append(8)

        elif dataset_name == 'CAMS':
            if 'none' in output_an.lower():
                output_label.append(0)
            elif 'bias' in output_an.lower():
                output_label.append(1)
            elif 'job' in output_an.lower():
                output_label.append(2)
            elif 'medication' in output_an.lower():
                output_label.append(3)
            elif 'relation' in output_an.lower():
                output_label.append(4)
            elif 'alienation' in output_an.lower():
                output_label.append(5)
            else:
                raise Exception('Wrong label in predictions for {}'.format(dataset_name))

            if 'no causes' in ref_an.lower():
                golden_label.append(0)
            elif 'bias or abuse' in ref_an.lower():
                golden_label.append(1)
            elif 'jobs and career' in ref_an.lower():
                golden_label.append(2)
            elif 'medication' in ref_an.lower():
                golden_label.append(3)
            elif 'relationship' in ref_an.lower():
                golden_label.append(4)
            elif 'alienation' in ref_an.lower():
                golden_label.append(5)
    avg_accuracy = round(accuracy_score(golden_label, output_label) * 100, 2)
    weighted_f1 = round(f1_score(golden_label, output_label, average='weighted') * 100, 2)
    micro_f1 = round(f1_score(golden_label, output_label, average='micro') * 100, 2)
    macro_f1 = round(f1_score(golden_label, output_label, average='macro') * 100, 2)
    print("Dataset: {}, average acc:{}, weighted F1 {}, micro F1 {}, macro F1 {}".format(dataset_name,
                                                                                         avg_accuracy, weighted_f1,
                                                                                         micro_f1, macro_f1))

In [6]:
import pandas as pd

In [7]:
data = pd.read_csv("/content/Irf2.csv")

In [8]:
goldens = data['goldens'].to_list()
generated_text = data['generated_text'].to_list()

In [9]:
labels, num_labels = get_dict("Irf")

In [10]:
from transformers import AutoModel, AutoTokenizer

# Replace 'bert-base-uncased' with the name of the model you want
model_name = 'Tianlin668/Irf'

# Download the pre-trained model and tokenizer
mentalbert = BertForSequenceClassification.from_pretrained(model_name,num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(model_name)

(…)/Tianlin668/Irf/resolve/main/config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

(…)8/Irf/resolve/main/tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

(…)co/Tianlin668/Irf/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)Irf/resolve/main/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

(…)anlin668/Irf/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
print(labels)

{0: 'no', 1: 'yes'}


In [12]:
all_labels = []
for i in range(0, len(generated_text)):
    batch_data = generated_text[i]
    inputs = tokenizer(batch_data, return_tensors="pt", truncation=True, max_length=512)
    outputs = mentalbert(**inputs)[0]
    outputs = outputs.cpu().detach().numpy()
    all_labels.append(np.argmax(outputs))
print(all_labels)
final_labels = []
for num in all_labels:
    final_labels.append(labels[num])
calculate_f1(goldens, final_labels,"Irf")

[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0]
Dataset: Irf, average acc:74.0, weighted F1 73.09, micro F1 74.0, macro F1 67.02


TASK : 2 CORRECTNESS

In [13]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=64e03d80a01bd2b860460b0829840e1a7aba4c4cfe64a913461a7dd26a8a8eec
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [14]:
def rouge():
    rouge = evaluate.load('rouge')
    score_results = {}
    dname = "Irf"
    predictions = data['generated_text'].to_list()
    references = data['goldens'].to_list()
    result = rouge.compute(predictions=predictions, references=references)
    score_results[dname] = result
    return score_results


In [15]:
print(rouge())

{'Irf': {'rouge1': 0.5630138535641704, 'rouge2': 0.3140001120856981, 'rougeL': 0.39181923510493144, 'rougeLsum': 0.4818315241021617}}


In [16]:
def bleu():
    rouge = evaluate.load('bleu')
    score_results = {}
    dname = "Irf"
    predictions = data['generated_text'].to_list()
    references = data['goldens'].to_list()
    result = rouge.compute(predictions=predictions, references=references)
    score_results[dname] = result['bleu']
    return score_results


In [17]:
print(bleu())

{'Irf': 0.2793076359126411}


In [18]:
!git clone https://github.com/neulab/BARTScore

Cloning into 'BARTScore'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 220 (delta 18), reused 14 (delta 14), pack-reused 194
Receiving objects: 100% (220/220), 101.98 MiB | 12.83 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Updating files: 100% (192/192), done.


In [19]:

from BARTScore.bart_score import BARTScorer
def compute_bart_score(a,b):
    scorer = BARTScorer()
    score = scorer.score(a,b)
    return score


In [20]:
print(sum(compute_bart_score(generated_text,goldens))/100)

(…)k/bart-large-cnn/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)k/bart-large-cnn/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)rt-large-cnn/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(…)/bart-large-cnn/resolve/main/config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

(…)-cnn/resolve/main/generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

-2.665697922706604


In [21]:
pip install bert-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00


In [22]:
from bert_score import score

def calculate_bert_score(reference, candidate):

    P, R, F1 = score(candidate, reference, lang="en", verbose=True)

    return F1.mean().item()


reference_sentence = goldens
candidate_sentence = generated_text

bert_score = calculate_bert_score(reference_sentence,candidate_sentence)

print("BERTScore:", bert_score)


(…)o/roberta-large/resolve/main/config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

(…)co/roberta-large/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)co/roberta-large/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)oberta-large/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 5.61 seconds, 17.82 sentences/sec
BERTScore: 0.9087938070297241
